In [69]:
import pandas as pd
import os

#set file path
project_env = r"C:\Users\Finn Kopmann\Dokumente\Beruf und Finanzen\Fortbildung\data engineer\projects\case_study"

# Full path to the CSV files
file_path_crm = os.path.join(project_env, r"data\crm_data.csv")
file_path_web_tracking = os.path.join(project_env, r"data\web_tracking.csv")

# Load the CSV files
crm_df = pd.read_csv(file_path_crm)
web_tracking_df = pd.read_csv(file_path_web_tracking)

In [70]:
# Display the first few rows
crm_df.head()

,user_id,lead_status,company,sign_up_date,industry,country
0,137,Converted,Company_0,2024-03-09,Healthcare,US
1,163,Converted,Company_1,2024-03-01,Finance,UK
2,103,Prospect,Company_2,2024-03-02,Healthcare,France
3,163,Lost,Company_3,2024-03-01,SaaS,US
4,171,Lost,Company_4,2024-03-07,Healthcare,Germany


In [71]:
# Display the first few rows
web_tracking_df.head()

,user_id,session_id,timestamp,page,time_spent_sec,utm_source,utm_medium,device_type
0,151,s000,2024-03-07 12:55:00,/pricing,54,newsletter,referral,mobile
1,192,s001,2024-03-05 14:21:00,/landing,71,linkedin,email,desktop
2,114,s002,2024-03-01 14:25:00,/contact,143,facebook,cpc,desktop
3,171,s003,2024-03-06 16:56:00,/contact,293,newsletter,email,desktop
4,160,s004,2024-03-01 21:41:00,/contact,37,linkedin,organic,tablet


In [96]:
web_tracking_df.groupby('user_id').page.apply(list)

user_id
101    [/contact, /features, /landing, /landing]
102                         [/landing, /contact]
103              [/features, /contact, /contact]
105                                  [/features]
106                                  [/features]
                         ...                    
190                                  [/features]
191                         [/contact, /pricing]
192                         [/landing, /landing]
194                                  [/features]
199                        [/landing, /features]
Name: page, Length: 61, dtype: object

In [72]:
# Create Dimension Tables
lead_status_dim = crm_df[['lead_status']].drop_duplicates().reset_index(drop=True)
lead_status_dim['lead_status_id'] = lead_status_dim.index + 1


SyntaxError: incomplete input (766641423.py, line 2)

In [80]:
company_dim = crm_df[['company']].drop_duplicates().reset_index(drop=True)
company_dim['company_id'] = company_dim.index + 1

In [74]:
industry_dim = crm_df[['industry']].drop_duplicates().reset_index(drop=True)
industry_dim['industry_id'] = industry_dim.index + 1

In [75]:
country_dim = crm_df[['country']].drop_duplicates().reset_index(drop=True)
country_dim['country_id'] = country_dim.index + 1

In [93]:
# Sign-up time breakdown
crm_df['sign_up_datetime'] = pd.to_datetime(crm_df['sign_up_date'])
sign_up_time_dim = crm_df[['sign_up_datetime']].drop_duplicates().reset_index(drop=True)
sign_up_time_dim['sign_up_time_id'] = sign_up_time_dim.index + 1
sign_up_time_dim['sign_up_year'] = sign_up_time_dim['sign_up_datetime'].dt.year
sign_up_time_dim['sign_up_month'] = sign_up_time_dim['sign_up_datetime'].dt.month
sign_up_time_dim['sign_up_week'] = sign_up_time_dim['sign_up_datetime'].dt.isocalendar().week
sign_up_time_dim['sign_up_weekday'] = sign_up_time_dim['sign_up_datetime'].dt.weekday

In [92]:
# User Dimension Table
user_dim = crm_df.merge(company_dim, on="company")\
.merge(industry_dim, on="industry")\
.merge(country_dim, on="country")\
.merge(sign_up_time_dim, on="sign_up_datetime")\
.merge(lead_status_dim, on="lead_status")\
[["user_id","lead_status_id", "industry_id", "company_id", "country_id", "sign_up_time_id"]].drop_duplicates(subset='user_id')

In [85]:
# Prepare Session Data Dimensions
web_tracking_df["timestamp_datetime"] = pd.to_datetime(web_tracking_df['timestamp'])
datetime_dim = web_tracking_df[['timestamp_datetime']].drop_duplicates().reset_index(drop=True)
datetime_dim['datetime_id'] = datetime_dim.index + 1
datetime_dim['year'] = datetime_dim['timestamp_datetime'].dt.year
datetime_dim['month'] = datetime_dim['timestamp_datetime'].dt.month
datetime_dim['week'] = datetime_dim['timestamp_datetime'].dt.isocalendar().week
datetime_dim['weekday'] = datetime_dim['timestamp_datetime'].dt.weekday

In [86]:
page_dim = web_tracking_df[['page']].drop_duplicates().reset_index(drop=True)
page_dim['page_id'] = page_dim.index + 1

In [87]:
utm_source_dim = web_tracking_df[['utm_source']].drop_duplicates().reset_index(drop=True)
utm_source_dim['utm_source_id'] = utm_source_dim.index + 1

In [88]:
utm_medium_dim = web_tracking_df[['utm_medium']].drop_duplicates().reset_index(drop=True)
utm_medium_dim['utm_medium_id'] = utm_medium_dim.index + 1

In [89]:
device_type_dim = web_tracking_df[['device_type']].drop_duplicates().reset_index(drop=True)
device_type_dim['device_type_id'] = device_type_dim.index + 1

In [91]:
# Fact Table
fact_table = web_tracking_df.merge(user_dim, on="user_id")\
.merge(datetime_dim, on="timestamp_datetime")\
.merge(page_dim, on="page")\
.merge(utm_source_dim, on="utm_source")\
.merge(utm_medium_dim, on="utm_medium")\
.merge(device_type_dim, on="device_type")\
[['session_id', 'user_id', 'datetime_id', 'page_id', 'utm_source_id', 'utm_medium_id', 'device_type_id', 'time_spent_sec']]